### Data Cleaning

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1=pd.read_csv("schema_1_ontario_final.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'schema_1_ontario_final.csv'

In [ ]:
df1

In [ ]:
df2=pd.read_csv("schema_2_ontario_final.csv")

In [ ]:
df2

In [ ]:
df3=pd.read_csv("schema_3_ontario_final.csv")

In [ ]:
df3

In [ ]:
df1

In [ ]:
df1.week.unique()

#### DEDUPLICATION
##### Keep only the latest record per session to prevent double-counting individuals
##### The y row is the "Truth," and the n rows are the "Drafts" in is_most_recent
##### If you keep only y: You count one person with a heart condition. This is the truth—the most accurate snapshot of that person's health. If you keep both: You count two people. One is healthy; one has a heart condition. This is a lie; there is only one person.

In [ ]:
df1_clean = df1[df1['is_most_recent'] == 'y'].copy()

In [ ]:
df1_clean

#### STEP 2: TEMPORAL MAPPING
##### Weeks 13, 14, and 15 in this dataset correspond to 'April'

In [ ]:
week_to_month = {13: 'April', 14: 'April', 15: 'April'}
df1_clean['month'] = df1_clean['week'].map(week_to_month)

In [ ]:
df1_clean = df1_clean.drop(columns=['week'])

In [ ]:
df1_clean

#### STEP 3: AGE NORMALIZATION ---
##### Map S1 'over_60' to the 'age_1' categories used in Schema 3 dataset
##### Dataset Schema 2 & 3 are more detailed: They use specific brackets like 26-44, 45-64, and >65 So changed the over_60 column

In [ ]:
df1_clean['age_1'] = df1_clean['over_60'].map({'y': '>65', 'n': '<65'})

In [ ]:
df1_clean = df1_clean.drop(columns=['over_60'])

In [ ]:
df1_clean

#### STEP 4: BINARY STANDARDIZATION
##### Convert 'y'/'n' strings to 1/0 integers for health and exposure flags as you can’t do math on letters, Computers process numbers faster than text, machine learning compatibility

In [ ]:
binary_cols = [
    'probable', 'vulnerable', 'fever_chills_shakes', 
    'cough', 'shortness_of_breath', 'any_medical_conditions', 
    'travel_outside_canada', 'contact_with_illness', 'is_most_recent'
]

for col in binary_cols:
    if col in df1_clean.columns:
        df1_clean[col] = df1_clean[col].map({'y': 1, 'n': 0}).fillna(0).astype(int)

In [ ]:
df1_clean

#### STEP 5: FINAL REFINEMENT
##### Drop 'week' and 'over 60' columns that are now redundant and reorder 'month' to the front

In [ ]:
cols = ['month'] + [c for c in df1_clean.columns if c != 'month']
df1_clean = df1_clean[cols]

In [ ]:
df1_clean

#### Dataset Schema 2
##### Load the dataset 2

In [ ]:
df2

#### 1: DEDUPLICATION
##### Keep only the latest record per session

In [ ]:
df2_clean = df2[df2['is_most_recent'] == 'y'].copy()

#### STEP 2: TEMPORAL MAPPING
##### Schema 2 contains weeks 15 and 16, which fall in April

In [ ]:
df2.week.unique()

In [ ]:
week_to_month = {15: 'April', 16: 'April'}
df2_clean['month'] = df2_clean['week'].map(week_to_month)

In [ ]:
df1_clean

In [ ]:
df2_clean = df2_clean.drop(columns=['week'])

#### STEP 3: BINARY STANDARDIZATION
##### Convert 'y'/'n' to 1/0 for health and behavior flags

In [ ]:
binary_cols = [
    'probable', 'vulnerable', 'fever_chills_shakes', 
    'cough', 'shortness_of_breath', 'any_medical_conditions',
    'travel_outside_canada', 'contact_with_illness'
]

for col in binary_cols:
    if col in df2_clean.columns:
        df2_clean[col] = df2_clean[col].map({'y': 1, 'n': 0}).fillna(0).astype(int)

##### Move 'month' to the first column to match the other schemas

In [ ]:
cols = ['month'] + [c for c in df2_clean.columns if c != 'month']
df2_clean = df2_clean[cols]

In [ ]:
df2_clean

##### symptoms:	NaN changed to 'none' Reason is The person is asymptomatic.
##### conditions:	NaN	changed to 'none' Reason is The person has no pre-existing conditions.
##### ethnicity	NaN	changed to 'none' Reason is The person did not specify their ethnicity.
##### needs	NaN	changed to 'none' Reason is The person does not require emergency help.

In [ ]:
text_cols = ['symptoms', 'conditions', 'ethnicity', 'needs']

# Fill NaNs with 'none' to represent a negative or neutral response
for col in text_cols:
    if col in df2_clean.columns:
        df2_clean[col] = df2_clean[col].fillna('none')

In [ ]:
df2_clean.isnull().sum()

In [ ]:
df3

In [ ]:
df3.columns

In [ ]:
df3_clean =df3.copy()

In [ ]:
# STEP 2: HANDLE NaN VALUES
# Text-based columns often have NaNs when the user has nothing to report


In [ ]:
text_cols = ['symptoms', 'conditions', 'ethnicity']
for col in text_cols:
    if col in df3_clean.columns:
        df3_clean[col] = df3_clean[col].fillna('none')

In [ ]:
# STEP 3: BINARY ENCODING (1 and 0)
# List of all Yes/No columns present in Schema 3

In [ ]:
binary_cols = [
    'probable', 'vulnerable', 'fever_chills_shakes', 
    'cough', 'shortness_of_breath', 'any_medical_conditions',
    'travel_outside_canada', 'contact_with_illness'
]

for col in binary_cols:
    if col in df3_clean.columns:
        # Convert y/n to 1/0 and handle any remaining NaNs as 0
        df3_clean[col] = df3_clean[col].map({'y': 1, 'n': 0}).fillna(0).astype(int)

In [ ]:
# 4. Final Verification
# Ensure there are no leftover NaNs in critical columns
print(df3_clean[['month', 'age_1', 'cough', 'ethnicity']].isnull().sum())

# Save the cleaned dataset
#df3.to_csv('schema_3_cleaned.csv', index=False)

In [ ]:
df3_clean

In [ ]:
# 1. Define groups for different filling strategies
none_fill_cols = [
    'needs', 'mental_health_impact', 'travel_work_school', 
    'self_isolating', 'media_channels', 'financial_obligations_impact', 
    'tobacco_usage'
]

unknown_fill_cols = ['sex', 'age_1']

# 2. Apply the fills
# Fill behavioral/impact columns with 'none'
for col in none_fill_cols:
    if col in df3_clean.columns:
        df3_clean[col] = df3_clean[col].fillna('none')

# Fill demographic columns with 'unknown'
for col in unknown_fill_cols:
    if col in df3_clean.columns:
        df3_clean[col] = df3_clean[col].fillna('unknown')

In [ ]:
df3_clean

In [ ]:
# 1. Fill NaN values with 'none' first (very important!)
df3_clean['symptoms'] = df3_clean['symptoms'].fillna('none')

# 2. Split the semicolon-separated strings into a matrix of 1s and 0s
symptom_dummies = df3_clean['symptoms'].str.get_dummies(sep=';')

# 3. Combine this new matrix with your original dataset
# We prefix the columns with 'symp_' so we know where they came from
symptom_dummies = symptom_dummies.add_prefix('symp_')
df3_final = pd.concat([df3_clean, symptom_dummies], axis=1)

# 4. Optional: Drop the original text column to save space
df3_final = df3_final.drop(columns=['symptoms'])

In [ ]:
df3_final

In [ ]:
df3_clean['age_1']

In [ ]:
df3_final

In [ ]:
df3_final['covid_positive_clean'] = (
    df3_final['covid_positive']
    .astype(str)
    .str.strip()
    .str.lower()
)
df3_final['covid_positive_num'] = df3_final['covid_positive_clean'].map({
    'positively': 1,
    'positive': 1,
    'n': 0,
    'negatively': 0,
    'negative': 0
})
df3_final['covid_positive'] = df3_final['covid_positive_num'].astype('Int64')
df3_final.drop('covid_positive_num',axis=1, inplace=True)
df3_final.drop('covid_positive_clean',axis=1, inplace=True)

In [ ]:
df3_final.columns

In [ ]:
#df3_final.to_csv('cleaned_schema_3_data_Ela.csv', index=False)

In [ ]:
pip install openpyxl

In [ ]:
# Convert the dataframe to an Excel file
df3_final.to_excel('cleaned_schema_3_data.xlsx', index=False)

In [ ]:
df2_clean.to_excel('cleaned_schema_2_data.xlsx', index=False)

In [ ]:
df1_clean.to_excel('cleaned_schema_1_data.xlsx', index=False)

In [ ]:
df1_clean['age_binary'] = df1_clean['age_1'].map({'>65': 1, '<65': 0})

In [ ]:
df1_clean

In [ ]:
print(df2_clean['age_1'].unique())

In [ ]:
df2_clean['age_binary'] = df2_clean['age_1'].map({'>65': 1, '<65': 0})

In [ ]:
df2_clean

In [ ]:
df2_clean = pd.get_dummies(df2_clean, columns=['age_1'], dtype=int)

In [ ]:
df3_final['age_1']

In [ ]:
df3_final = pd.get_dummies(df3_final, columns=['age_1'], dtype=int)

In [ ]:
df3_final